In [1]:
pip install --upgrade polars[plot]==0.20.31

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2


In [2]:
PROJECT_ID = "bt-int-wod-masterclass-c3f3"
REGION = "EU"
DATASET_ID = "data_beans_curated_local"
TABLE_NAME = "customer_review_embedded_laurenz"
CONNECTION_NAME = "vertex-ai"

In [28]:
import bigframes.pandas as bpd
from google.cloud import bigquery

import polars as pl

from bokeh.io import show, save
import hvplot
import hvplot.polars

In [4]:
client = bigquery.Client(project=PROJECT_ID, location=REGION)

In [5]:
# copy embedded customer reviews into a polars dataframe
query = (
    f"SELECT * FROM `{DATASET_ID}.{TABLE_NAME}` r")
query_job = client.query(query)  # API request
# copy table into polars dataframe
customer_review_embedded_raw = pl.from_arrow(query_job.result().to_arrow())


In [6]:
customer_review_embedded_raw

rowid,text_embedding,statistics,ml_embed_text_status,content
i64,list[f64],str,str,str
7461,"[0.048845, -0.020206, … 0.054143]","""{""token_count"":52,""truncated"":…","""""","""I'm a coffee fanatic and I don…"
7549,"[0.047678, -0.034145, … 0.035929]","""{""token_count"":55,""truncated"":…","""""","""I always get my coffee from he…"
7559,"[0.004317, -0.034291, … 0.067955]","""{""token_count"":55,""truncated"":…","""""","""I had never been to a location…"
7560,"[0.04532, -0.018647, … 0.070964]","""{""token_count"":55,""truncated"":…","""""","""This place has a great selecti…"
7603,"[0.053228, -0.023868, … 0.047716]","""{""token_count"":56,""truncated"":…","""""","""I stopped by this coffee truck…"
…,…,…,…,…
7492,"[0.041095, -0.065384, … 0.047765]","""{""token_count"":53,""truncated"":…","""""","""I think it is overpriced, but …"
7512,"[0.029933, -0.011228, … 0.050344]","""{""token_count"":53,""truncated"":…","""""","""The staff was friendly and hel…"
7500,"[0.025881, -0.038813, … 0.057246]","""{""token_count"":53,""truncated"":…","""""","""The staff were very friendly a…"


In [7]:
#convert the embedding array column to individual columns, one for each array item, using a list comprehension
customer_review_embedded = customer_review_embedded_raw.with_columns(
    (pl.col("text_embedding").cast(pl.Array(pl.Float64, width=768)).arr.get(i).alias(f"embedding_{i}") for i in range(768))
)

<ipython-input-7-cfdd6fc7e06a>:3: DeprecationWarning: The `width` parameter for `Array` is deprecated. Use `shape` instead.
  (pl.col("text_embedding").cast(pl.Array(pl.Float64, width=768)).arr.get(i).alias(f"embedding_{i}") for i in range(768))


In [8]:
customer_review_embedded

rowid,text_embedding,statistics,ml_embed_text_status,content,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15,embedding_16,embedding_17,embedding_18,embedding_19,embedding_20,embedding_21,embedding_22,embedding_23,embedding_24,embedding_25,embedding_26,embedding_27,embedding_28,embedding_29,embedding_30,embedding_31,…,embedding_731,embedding_732,embedding_733,embedding_734,embedding_735,embedding_736,embedding_737,embedding_738,embedding_739,embedding_740,embedding_741,embedding_742,embedding_743,embedding_744,embedding_745,embedding_746,embedding_747,embedding_748,embedding_749,embedding_750,embedding_751,embedding_752,embedding_753,embedding_754,embedding_755,embedding_756,embedding_757,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
i64,list[f64],str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
7461,"[0.048845, -0.020206, … 0.054143]","""{""token_count"":52,""truncated"":…","""""","""I'm a coffee fanatic and I don…",0.048845,-0.020206,-0.018818,-0.047474,0.038713,0.045697,-0.017239,-0.042988,-0.006626,0.029441,0.000612,0.004637,0.010675,-0.004681,-0.012631,0.020612,-0.011457,0.000392,-0.009434,-0.058769,0.000176,-0.002464,0.04422,-0.021724,0.006449,-0.054143,0.037962,-0.005147,-0.06251,0.011067,-0.086287,0.066934,…,0.003121,-0.048528,0.014404,-0.048726,0.091999,-0.019359,0.009935,-0.01678,-0.070496,-0.016119,-0.049654,0.074149,0.081228,0.067097,0.009673,0.023677,-0.016921,-0.02818,-0.040767,-0.016931,-0.015579,-0.045161,0.084795,0.023857,0.018486,0.009689,0.009537,0.024732,0.032967,-0.041192,0.04709,-0.039201,0.049214,0.04284,0.027583,0.008554,0.054143
7549,"[0.047678, -0.034145, … 0.035929]","""{""token_count"":55,""truncated"":…","""""","""I always get my coffee from he…",0.047678,-0.034145,-0.025936,-0.054205,0.044886,0.051056,-0.0296,-0.052977,0.005275,-0.011844,-0.015338,0.001637,-0.004763,-0.023152,-0.041484,-0.006018,-0.027571,-0.007105,-0.031696,-0.053676,-0.021392,0.01215,0.073769,-0.019532,0.008011,-0.047853,-0.007672,-0.013809,-0.040439,-0.012788,-0.068001,0.040584,…,0.001261,-0.025588,0.03546,-0.080545,0.087602,-0.046749,0.018271,-0.045936,-0.074644,-0.045385,-0.02517,0.062332,0.052306,0.074276,-0.024718,0.03709,-0.001617,-0.02995,-0.000615,0.008106,0.0111,-0.020192,0.023027,0.006353,0.022236,-0.00609,0.028793,0.027707,0.027476,-0.026624,0.065381,-0.041634,0.018789,0.019633,0.002239,-0.02994,0.035929
7559,"[0.004317, -0.034291, … 0.067955]","""{""token_count"":55,""truncated"":…","""""","""I had never been to a location…",0.004317,-0.034291,-0.020192,-0.006772,0.033597,0.090561,-0.002594,-0.035049,0.007243,-0.023208,-0.039742,0.003135,0.028551,-0.036963,-0.05121,0.002579,-0.033508,0.010342,0.006456,-0.047412,-0.007196,0.005565,0.069751,-0.036811,0.005051,-0.062901,0.004829,-0.038101,-0.029347,0.030128,-0.046597,0.07112,…,0.003918,-0.000431,0.031944,-0.04806,0.063712,-0.04396,-0.014613,-0.023835,-0.086876,-0.025749,-0.008571,0.063344,0.035737,0.086655,-0.035771,0.036079,0.009569,-0.005663,0.01644,0.018999,-0.033631,-0.008642,0.022383,0.013164,0.02605,0.002983,0.054093,0.011348,0.047733,-0.024122,0.052828,-0.022036,0.024124,0.024854,0.007793,-0.020556,0.067955
7560,"[0.04532, -0.018647, … 0.070964]","""{""token_count"":55,""truncated"":…","""""","""This place has a great selecti…",0.04532,-0.018647,-0.033229,-0.054886,0.066516,0.051332,-0.009663,-0.049191,0.01406,-0.004939,0.015234,-0.022976,0.015583,-0.030902,-0.022509,-0.011805,-0.020288,0.010996,-0.013753,-0.046997,-0.011644,0.001977,0.054376,-0.013521,-0.004046,-0.060478,0.015322,-0.0033

In [9]:
# Set BigQuery DataFrames options
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = REGION

In [10]:
# convert customer_review_embedded to Bigframe (only the embedding columns)
customer_review_embedded_bf = bpd.read_pandas(customer_review_embedded.select(pl.selectors.matches(r"^embedding_")).to_pandas())

In [11]:
customer_review_embedded_bf

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,0.048845,-0.020206,-0.018818,-0.047474,0.038713,0.045697,-0.017239,-0.042988,-0.006626,0.029441,...,0.024732,0.032967,-0.041192,0.04709,-0.039201,0.049214,0.04284,0.027583,0.008554,0.054143
1,0.047678,-0.034145,-0.025936,-0.054205,0.044886,0.051056,-0.0296,-0.052977,0.005275,-0.011844,...,0.027707,0.027476,-0.026624,0.065381,-0.041634,0.018789,0.019633,0.002239,-0.02994,0.035929
2,0.004317,-0.034291,-0.020192,-0.006772,0.033597,0.090561,-0.002594,-0.035049,0.007243,-0.023208,...,0.011348,0.047733,-0.024122,0.052828,-0.022036,0.024124,0.024854,0.007793,-0.020556,0.067955
3,0.04532,-0.018647,-0.033229,-0.054886,0.066516,0.051332,-0.009663,-0.049191,0.01406,-0.004939,...,0.008287,0.037472,-0.026117,0.055417,-0.043377,0.01465,0.032689,-0.014966,-0.011814,0.070964
4,0.053228,-0.023868,-0.049221,-0.061731,0.04267,0.062132,0.000897,-0.023412,-0.008498,0.014876,...,0.002963,0.034895,-0.037654,0.034019,-0.048664,0.018646,0.013108,-0.010859,-0.00461,0.047716
5,0.026698,-0.013233,-0.033684,-0.033508,0.038089,0.048833,-0.009421,-0.033227,0.030092,-0.003076,...,0.02108,0.066222,-0.020456,0.034009,-0.021611,0.037692,0.014901,0.016053,-0.020357,0.051574
6,0.040598,-0.013675,0.005653,-0.024912,0.033753,0.058193,-0.000557,-0.043817,0.009429,-0.005622,...,0.007196,0.037509,-0.01499,0.048047,-0.038213,0.025821,0.030311,0.004337,-0.009171,0.058013
7,0.024971,-0.032426,-0.04778,-0.04042,0.033208,0.085341,-0.008933,-0.017382,0.003807,-0.001902,...,0.00323,0.043043,-0.012381,0.023846,-0.027226,0.041813,0.030772,-0.005052,-0.018343,0.035575
8,0.028715,-0.004995,-0.058901,-0.040589,0.027722,0.056442,0.010439,-0.062266,-0.010051,-0.009421,...,0.005806,0.032287,-0.034392,0.039063,-0.04518,0.031915,0.024755,-0.008787,-0.026162,0.049303
9,0.025832,-0.019892,-0.022126,-0.019593,0.033527,0.046461,-0.004793,-0.041055,0.011118,-0.01999,...,-0.012599,0.049826,-0.052484,0.046917,-0.025565,0.035416,0.008906,0.024642,-0.027964,0.071153


In [12]:
import bigframes.ml.decomposition as dc

In [13]:
pca = dc.PCA(n_components=10)

In [14]:
pca.fit(X=customer_review_embedded_bf)

PCA(n_components=10)

In [18]:
pca.explained_variance_ratio_.sort_values(by="explained_variance_ratio", ascending=False)

,principal_component_id,explained_variance_ratio
2,0,0.134559
4,1,0.066765
6,2,0.059778
7,3,0.051538
9,4,0.04622
3,5,0.036796
1,6,0.034411
0,7,0.031167
8,8,0.021263
5,9,0.020868


In [22]:
coords_after_pca = pl.from_pandas(pca.predict(X=customer_review_embedded_bf).to_pandas())

In [23]:
coords_after_pca

principal_component_1,principal_component_2,principal_component_3,principal_component_4,principal_component_5,principal_component_6,principal_component_7,principal_component_8,principal_component_9,principal_component_10
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.925665,-1.911317,-1.724364,10.259554,2.721579,9.261119,3.667532,-3.16673,3.634138,2.770968
-6.222728,-1.066516,-3.318703,0.487521,1.067717,8.312714,9.98865,-7.468767,-0.268469,5.280275
-7.03107,-1.128108,-5.63462,-3.592317,-1.068499,1.129989,4.203582,3.246923,1.312858,8.254532
-11.235043,3.857524,-2.780472,6.032825,3.378198,5.796707,6.46881,-0.389664,4.151447,2.776398
-11.979665,-7.670064,9.700599,3.773867,0.386152,-2.512794,2.322554,2.551778,2.14362,-3.621871
…,…,…,…,…,…,…,…,…,…
7.786922,-0.19147,-4.769679,9.767386,1.240049,-3.50724,2.795626,-1.30248,-2.05993,4.223208
-10.975461,1.634239,-0.615791,-0.237953,-2.129387,-0.11829,4.958987,6.613494,1.03222,-3.973353
-11.921846,-0.881251,-8.341742,-1.169255,3.026805,3.215227,3.083298,7.348188,1.074097,1.010134


In [24]:
coords_after_pca = coords_after_pca.with_columns(content=pl.Series(customer_review_embedded.select("content")))

In [29]:
scatterplot = coords_after_pca.plot(kind="scatter", x="principal_component_1", y="principal_component_2", hover_cols=["content"], width=1000, height=600)
scatterplot_bokeh = hvplot.render(scatterplot, backend="bokeh")
show(scatterplot_bokeh)